In [12]:
import datetime
from datetime import timedelta
import blpapi
from blpapi import SessionOptions, Session

import blpapi
from blpapi import SessionOptions, Session
import datetime

import os
import csv
import datetime
from datetime import timedelta
import blpapi
from blpapi import SessionOptions, Session

Get data from Bloomberg

In [13]:


def build_option_ticker(ticker_base, expiry, strike, call_or_put):
    """
    构建 Bloomberg 的期权 ticker 字符串
    """
    expiry_formatted = datetime.datetime.strptime(expiry, "%Y-%m-%d").strftime("%m/%d/%y")
    return f"{ticker_base} {expiry_formatted} {call_or_put.upper()}{int(strike)} Equity"

def get_option_price_and_iv(ticker, expiry, strike, call_or_put, start_date, end_date, price_field='PX_LAST', iv_field='IVOL_MID'):
    """
    输入：股票ticker、到期日、行权价、Call/Put、时间范围
    输出：历史价格和隐含波动率字典，格式 {date: {'price': price, 'iv': iv}}
    """
    option_ticker = build_option_ticker(ticker, expiry, strike, call_or_put)
    
    options = SessionOptions()
    options.setServerHost('localhost')
    options.setServerPort(8194)

    session = Session(options)
    if not session.start():
        raise RuntimeError("无法连接 Bloomberg")

    if not session.openService("//blp/refdata"):
        raise RuntimeError("无法打开服务")

    refDataService = session.getService("//blp/refdata")
    request = refDataService.createRequest("HistoricalDataRequest")

    request.getElement("securities").appendValue(option_ticker)
    # 请求价格和隐含波动率字段
    request.getElement("fields").appendValue(price_field)
    request.getElement("fields").appendValue(iv_field)
    request.set("startDate", start_date.replace('-', ''))
    request.set("endDate", end_date.replace('-', ''))
    request.set("periodicitySelection", "DAILY")

    session.sendRequest(request)

    data = {}
    while True:
        ev = session.nextEvent()
        for msg in ev:
            if msg.hasElement("securityData"):
                securityData = msg.getElement("securityData")
                fieldDataArray = securityData.getElement("fieldData")
                for i in range(fieldDataArray.numValues()):
                    record = fieldDataArray.getValue(i)
                    date = record.getElementAsDatetime("date").strftime('%Y-%m-%d')
                    price = None
                    iv = None
                    if record.hasElement(price_field):
                        price = record.getElementAsFloat(price_field)
                    if record.hasElement(iv_field):
                        iv = record.getElementAsFloat(iv_field)
                    data[date] = {'price': price, 'iv': iv}
        if ev.eventType() == blpapi.Event.RESPONSE:
            break

    return data


# ✅ 示例用法
if __name__ == "__main__":
    result = get_option_price_and_iv(
        ticker="AAPL US",
        expiry="2020-01-17",
        strike=300,
        call_or_put='C',
        start_date="2020-01-01",
        end_date="2020-01-17",
        price_field='PX_LAST',
        iv_field = 'IVOL_MID'
    )

    
    for date, values in result.items():
        print(f"{date} : Price={values['price']}, IV={values['iv']}")

# Expected output

# 2020-01-02 : Price=6.6, IV=25.585
# 2020-01-03 : Price=4.65, IV=24.585
# 2020-01-06 : Price=5.15, IV=24.874
# 2020-01-07 : Price=4.25, IV=24.784
# 2020-01-08 : Price=6.3, IV=23.698
# 2020-01-09 : Price=11.1, IV=26.144
# 2020-01-10 : Price=11.74, IV=28.288
# 2020-01-13 : Price=17.55, IV=36.551
# 2020-01-14 : Price=13.1, IV=36.565
# 2020-01-15 : Price=11.53, IV=36.525
# 2020-01-16 : Price=15.4, IV=38.344
# 2020-01-17 : Price=18.61, IV=None

2020-01-02 : Price=6.6, IV=25.585
2020-01-03 : Price=4.65, IV=24.585
2020-01-06 : Price=5.15, IV=24.874
2020-01-07 : Price=4.25, IV=24.784
2020-01-08 : Price=6.3, IV=23.698
2020-01-09 : Price=11.1, IV=26.144
2020-01-10 : Price=11.74, IV=28.288
2020-01-13 : Price=17.55, IV=36.551
2020-01-14 : Price=13.1, IV=36.565
2020-01-15 : Price=11.53, IV=36.525
2020-01-16 : Price=15.4, IV=38.344
2020-01-17 : Price=18.61, IV=None


Find the longest data for apple 

In [14]:
def generate_candidate_expiries(start_date, end_date="2025-07-01"):
    """
    生成候选的期权到期日期（每周五）
    """
    expiry_start = datetime.datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=30)
    expiry_end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    
    candidate_expiries = []
    current_date = expiry_start
    while current_date <= expiry_end:
        if current_date.weekday() == 4:  # 周五
            candidate_expiries.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    
    return candidate_expiries


In [ ]:
def find_longest_history_option(ticker, start_date, end_date, strikes, call_or_put='C'):
    """
    寻找具有最长历史数据的期权合约
    """
    best_result = None
    candidate_expiries = generate_candidate_expiries(start_date)

    for expiry in candidate_expiries:
        for strike in strikes:
            try:
                result = get_option_price_and_iv(
                    ticker=ticker,
                    expiry=expiry,
                    strike=strike,
                    call_or_put=call_or_put,
                    start_date=start_date,
                    end_date=end_date,
                    price_field='PX_LAST',
                    iv_field='IVOL_MID'
                )
                
                if result:
                    dates = sorted(result.keys())
                    length = len(dates)
                    if best_result is None or length > best_result['length']:
                        best_result = {
                            'expiry': expiry,
                            'strike': strike,
                            'length': length,
                            'data_start': dates[0],
                            'data_end': dates[-1]
                        }
            except Exception as e:
                print(f"跳过 {expiry} strike={strike}: {e}")
    
    return best_result

def print_result(result):
    """
    打印结果
    """
    if result:
        print("✅ 最长历史数据期权:")
        print(f"Expiry:     {result['expiry']}")
        print(f"Strike:     {result['strike']}")
        print(f"Length:     {result['length']} days")
        print(f"Data Start: {result['data_start']}")
        print(f"Data End:   {result['data_end']}")
    else:
        print("❌ 未找到任何有效期权数据")

if __name__ == "__main__":
    # 配置参数
    CONFIG = {
        'ticker': "AAPL US",
        'call_or_put': 'C',
        'strikes': [150, 200, 250, 300],
        'start_date': "2024-07-01",
        'end_date': "2025-07-01"
    }
    
    # 执行查找
    result = find_longest_history_option(
        ticker=CONFIG['ticker'],
        start_date=CONFIG['start_date'],
        end_date=CONFIG['end_date'],
        strikes=CONFIG['strikes'],
        call_or_put=CONFIG['call_or_put']
    )
    
    # 打印结果
    print_result(result)


✅ 最长历史数据期权:
Expiry:     2025-06-20
Strike:     200
Length:     244 days
Data Start: 2024-07-01
Data End:   2025-06-20


Save the longest as csv file 

In [15]:
import blpapi
from blpapi import SessionOptions, Session
import datetime
import pandas as pd

ticker_base = "AAPL US"

def get_historical_option_data(ticker_base="AAPL US", 
                             expiry="2025-06-20",
                             strike=200,
                             call_or_put='C',
                             start_date="2024-07-01",
                             end_date="2025-06-20"):
    """
    获取指定期权的历史价格和隐含波动率数据
    """
    # 获取数据
    data = get_option_price_and_iv(
        ticker=ticker_base,
        expiry=expiry,
        strike=strike,
        call_or_put=call_or_put,
        start_date=start_date,
        end_date=end_date,
        price_field='PX_LAST',
        iv_field='IVOL_MID'
    )
    
    # 转换为DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')
    df.index = pd.to_datetime(df.index)
    df.index.name = 'date'
    df.columns = ['option_price', 'implied_volatility']
    
    # 基本统计信息
    print("数据概要:")
    print(f"数据起始日: {df.index.min().strftime('%Y-%m-%d')}")
    print(f"数据结束日: {df.index.max().strftime('%Y-%m-%d')}")
    print(f"总交易天数: {len(df)}")
    print("\n基本统计:")
    print(df.describe())
    
    return df

if __name__ == "__main__":
    # 使用之前找到的最长期限期权参数
    df = get_historical_option_data(
        ticker_base="AAPL US",
        expiry="2025-06-20",
        strike=200,
        call_or_put='C',
        start_date="2024-07-01",
        end_date="2025-06-20"
    )
    
    # 保存到CSV文件
    filename = f"X_{ticker_base}.csv"
    df.to_csv(filename)
    print(f"\n数据已保存至: {filename}")
    
    # 显示前几行数据
    print("\n数据预览:")
    print(df.head())


数据概要:
数据起始日: 2024-07-01
数据结束日: 2025-06-20
总交易天数: 244

基本统计:
       option_price  implied_volatility
count    244.000000          243.000000
mean      34.003443           28.267012
std       15.102613            3.754795
min        0.250000           23.219000
25%       25.070000           25.618000
50%       37.825000           27.015000
75%       43.880000           29.839500
max       65.450000           44.157000

数据已保存至: X_AAPL US.csv

数据预览:
            option_price  implied_volatility
date                                        
2024-07-01         35.26              23.545
2024-07-02         38.00              23.915
2024-07-03         39.00              24.002
2024-07-05         43.00              24.267
2024-07-08         44.10              24.147


Find several option for apple to hedge

In [16]:
def find_full_coverage_options(ticker, start_date, end_date, strikes, call_or_put='C', min_contracts=200):
    """
    找出一组期权，组合在一起能完整覆盖指定时间范围，
    并且数量不少于 min_contracts（默认200个），每个合约至少距离到期 20 天。
    """
    candidate_expiries = generate_candidate_expiries(start_date, end_date="2025-12-31")
    selected_options = []
    covered_days = set()

    # 构造目标时间范围的所有天
    target_days = set()
    d_start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    d_end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    delta = timedelta(days=1)
    d = d_start
    while d <= d_end:
        target_days.add(d.strftime("%Y-%m-%d"))
        d += delta

    for expiry in candidate_expiries:
        expiry_dt = datetime.datetime.strptime(expiry, "%Y-%m-%d")
        for strike in strikes:
            try:
                result = get_option_price_and_iv(
                    ticker=ticker,
                    expiry=expiry,
                    strike=strike,
                    call_or_put=call_or_put,
                    start_date=start_date,
                    end_date=end_date,
                    price_field='PX_LAST',
                    iv_field='IVOL_MID'
                )

                if result:
                    option_days = set(result.keys())
                    overlap = option_days & target_days - covered_days

                    if overlap:
                        data_start_str = min(option_days)
                        data_end_str = max(option_days)
                        data_start_dt = datetime.datetime.strptime(data_start_str, "%Y-%m-%d")

                        # ➕ 至少20天距离到期
                        if (expiry_dt - data_start_dt).days < 20:
                            print(f"⏩ 跳过 {expiry} {strike}：距离到期日 < 20 天")
                            continue

                        # 添加到选中列表
                        selected_options.append({
                            'expiry': expiry,
                            'strike': strike,
                            'length': len(result),
                            'data_start': data_start_str,
                            'data_end': data_end_str
                        })

                        covered_days.update(overlap)
                        print(f"✅ 收录: Expiry={expiry}, Strike={strike}, 覆盖 {len(overlap)} 新天数")

            except Exception as e:
                print(f"⚠️ 跳过 {expiry} {strike}: {e}")

        # 🔄 每次外层循环后检查是否满足退出条件
        if covered_days == target_days and len(selected_options) >= min_contracts:
            print(f"🎯 已完成全覆盖，并满足数量要求：{len(selected_options)} 合约")
            return selected_options

    print(f"⚠️ 警告：最终只找到 {len(selected_options)} 个合约")
    return selected_options


In [21]:
def save_option_data_to_csv(ticker_base, options, call_or_put, start_date, end_date, output_dir="ticker_hedge_data"):
    """
    批量保存期权历史价格和IV数据到CSV文件
    """
    os.makedirs(output_dir, exist_ok=True)
    success = 0
    fail = 0

    for opt in options:
        expiry = opt['expiry']
        strike = opt['strike']
        try:
            data = get_option_price_and_iv(
                ticker=ticker_base,
                expiry=expiry,
                strike=strike,
                call_or_put=call_or_put,
                start_date=start_date,
                end_date=end_date
            )

            if not data:
                print(f"⚠️ 无数据: {expiry} {strike} {call_or_put}")
                fail += 1
                continue

            strike_str = str(int(strike)) if float(strike).is_integer() else str(strike).replace('.', '_')
            filename = f"{ticker_base.replace(' ', '_')}_{expiry}_{call_or_put.upper()}{strike_str}.csv"
            filepath = os.path.join(output_dir, filename)

            with open(filepath, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["date", "price", "iv"])
                for date in sorted(data.keys()):
                    price = data[date]['price']
                    iv = data[date]['iv']
                    writer.writerow([date, price, iv])

            print(f"✅ 保存成功: {filename}")
            success += 1
        except Exception as e:
            print(f"❌ 错误跳过 {expiry} {strike}: {e}")
            fail += 1

    print(f"\n📦 保存完成：成功 {success} 个，失败 {fail} 个，输出目录：{output_dir}")

if __name__ == "__main__":
    CONFIG = {
        'ticker': "AAPL US",
        'call_or_put': 'C',
        'strikes': list(range(170, 260, 5)),
        'start_date': "2024-07-01",
        'end_date': "2025-07-01"
    }

    selected = find_full_coverage_options(
        ticker=CONFIG['ticker'],
        start_date=CONFIG['start_date'],
        end_date=CONFIG['end_date'],
        strikes=CONFIG['strikes'],
        call_or_put=CONFIG['call_or_put'],
        min_contracts=500
    )

    save_option_data_to_csv(
        ticker_base=CONFIG['ticker'],
        options=selected,
        call_or_put=CONFIG['call_or_put'],
        start_date=CONFIG['start_date'],
        end_date=CONFIG['end_date'],
        output_dir="ticker_hedge_data"
    )

KeyboardInterrupt: 

For morgan: price range from 210 to 290

In [22]:
def generate_candidate_expiries(start_date, end_date="2025-07-01"):
    """
    生成候选的期权到期日期（每周五）
    """
    expiry_start = datetime.datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=30)
    expiry_end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    
    candidate_expiries = []
    current_date = expiry_start
    while current_date <= expiry_end:
        if current_date.weekday() == 4:  # 周五
            candidate_expiries.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    
    return candidate_expiries


In [23]:
def find_longest_history_option(ticker, start_date, end_date, strikes, call_or_put='C'):
    """
    寻找具有最长历史数据的期权合约
    """
    best_result = None
    candidate_expiries = generate_candidate_expiries(start_date)

    for expiry in candidate_expiries:
        for strike in strikes:
            try:
                result = get_option_price_and_iv(
                    ticker=ticker,
                    expiry=expiry,
                    strike=strike,
                    call_or_put=call_or_put,
                    start_date=start_date,
                    end_date=end_date,
                    price_field='PX_LAST',
                    iv_field='IVOL_MID'
                )
                
                if result:
                    dates = sorted(result.keys())
                    length = len(dates)
                    if best_result is None or length > best_result['length']:
                        best_result = {
                            'expiry': expiry,
                            'strike': strike,
                            'length': length,
                            'data_start': dates[0],
                            'data_end': dates[-1]
                        }
            except Exception as e:
                print(f"跳过 {expiry} strike={strike}: {e}")
    
    return best_result

def print_result(result):
    """
    打印结果
    """
    if result:
        print("✅ 最长历史数据期权:")
        print(f"Expiry:     {result['expiry']}")
        print(f"Strike:     {result['strike']}")
        print(f"Length:     {result['length']} days")
        print(f"Data Start: {result['data_start']}")
        print(f"Data End:   {result['data_end']}")
    else:
        print("❌ 未找到任何有效期权数据")

if __name__ == "__main__":
    # 配置参数
    CONFIG = {
        'ticker': "JPM US",
        'call_or_put': 'C',
        'strikes': [200, 225, 250, 275, 300],
        'start_date': "2024-07-01",
        'end_date': "2025-07-01"
    }
    
    # 执行查找
    result = find_longest_history_option(
        ticker=CONFIG['ticker'],
        start_date=CONFIG['start_date'],
        end_date=CONFIG['end_date'],
        strikes=CONFIG['strikes'],
        call_or_put=CONFIG['call_or_put']
    )
    
    # 打印结果
    print_result(result)


❌ 未找到任何有效期权数据


In [ ]:
import blpapi
from blpapi import SessionOptions, Session
import datetime
import pandas as pd

ticker_base = "JPM US"

def get_historical_option_data(ticker_base=ticker_base, 
                             expiry="2025-06-20",
                             strike=200,
                             call_or_put='C',
                             start_date="2024-07-01",
                             end_date="2025-06-20"):
    """
    获取指定期权的历史价格和隐含波动率数据
    """
    # 获取数据
    data = get_option_price_and_iv(
        ticker=ticker_base,
        expiry=expiry,
        strike=strike,
        call_or_put=call_or_put,
        start_date=start_date,
        end_date=end_date,
        price_field='PX_LAST',
        iv_field='IVOL_MID'
    )
    
    # 转换为DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')
    df.index = pd.to_datetime(df.index)
    df.index.name = 'date'
    df.columns = ['option_price', 'implied_volatility']
    
    # 基本统计信息
    print("数据概要:")
    print(f"数据起始日: {df.index.min().strftime('%Y-%m-%d')}")
    print(f"数据结束日: {df.index.max().strftime('%Y-%m-%d')}")
    print(f"总交易天数: {len(df)}")
    print("\n基本统计:")
    print(df.describe())
    
    return df

if __name__ == "__main__":
    # 使用之前找到的最长期限期权参数
    df = get_historical_option_data(
        ticker_base=ticker_base,
        expiry="2025-06-20",
        strike=200,
        call_or_put='C',
        start_date="2024-07-01",
        end_date="2025-06-20"
    )
    
    # 保存到CSV文件
    filename = f"X_{ticker_base}_200.csv"
    df.to_csv(filename)
    print(f"\n数据已保存至: {filename}")
    
    # 显示前几行数据
    print("\n数据预览:")
    print(df.head())


In [ ]:
def find_full_coverage_options(ticker, start_date, end_date, strikes, call_or_put='C', min_contracts=200):
    """
    找出一组期权，组合在一起能完整覆盖指定时间范围，
    并且数量不少于 min_contracts（默认200个），每个合约至少距离到期 20 天。
    """
    candidate_expiries = generate_candidate_expiries(start_date, end_date="2025-12-31")
    selected_options = []
    covered_days = set()

    # 构造目标时间范围的所有天
    target_days = set()
    d_start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    d_end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    delta = timedelta(days=1)
    d = d_start
    while d <= d_end:
        target_days.add(d.strftime("%Y-%m-%d"))
        d += delta

    for expiry in candidate_expiries:
        expiry_dt = datetime.datetime.strptime(expiry, "%Y-%m-%d")
        for strike in strikes:
            try:
                result = get_option_price_and_iv(
                    ticker=ticker,
                    expiry=expiry,
                    strike=strike,
                    call_or_put=call_or_put,
                    start_date=start_date,
                    end_date=end_date,
                    price_field='PX_LAST',
                    iv_field='IVOL_MID'
                )

                if result:
                    option_days = set(result.keys())
                    overlap = option_days & target_days - covered_days

                    if overlap:
                        data_start_str = min(option_days)
                        data_end_str = max(option_days)
                        data_start_dt = datetime.datetime.strptime(data_start_str, "%Y-%m-%d")

                        # ➕ 至少20天距离到期
                        if (expiry_dt - data_start_dt).days < 20:
                            print(f"⏩ 跳过 {expiry} {strike}：距离到期日 < 20 天")
                            continue

                        # 添加到选中列表
                        selected_options.append({
                            'expiry': expiry,
                            'strike': strike,
                            'length': len(result),
                            'data_start': data_start_str,
                            'data_end': data_end_str
                        })

                        covered_days.update(overlap)
                        print(f"✅ 收录: Expiry={expiry}, Strike={strike}, 覆盖 {len(overlap)} 新天数")

            except Exception as e:
                print(f"⚠️ 跳过 {expiry} {strike}: {e}")

        # 🔄 每次外层循环后检查是否满足退出条件
        if covered_days == target_days and len(selected_options) >= min_contracts:
            print(f"🎯 已完成全覆盖，并满足数量要求：{len(selected_options)} 合约")
            return selected_options

    print(f"⚠️ 警告：最终只找到 {len(selected_options)} 个合约")
    return selected_options


In [ ]:
def save_option_data_to_csv(ticker_base, options, call_or_put, start_date, end_date, output_dir="ticker_hedge_data"):
    """
    批量保存期权历史价格和IV数据到CSV文件
    """
    os.makedirs(output_dir, exist_ok=True)
    success = 0
    fail = 0

    for opt in options:
        expiry = opt['expiry']
        strike = opt['strike']
        try:
            data = get_option_price_and_iv(
                ticker=ticker_base,
                expiry=expiry,
                strike=strike,
                call_or_put=call_or_put,
                start_date=start_date,
                end_date=end_date
            )

            if not data:
                print(f"⚠️ 无数据: {expiry} {strike} {call_or_put}")
                fail += 1
                continue

            strike_str = str(int(strike)) if float(strike).is_integer() else str(strike).replace('.', '_')
            filename = f"{ticker_base.replace(' ', '_')}_{expiry}_{call_or_put.upper()}{strike_str}.csv"
            filepath = os.path.join(output_dir, filename)

            with open(filepath, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["date", "price", "iv"])
                for date in sorted(data.keys()):
                    price = data[date]['price']
                    iv = data[date]['iv']
                    writer.writerow([date, price, iv])

            print(f"✅ 保存成功: {filename}")
            success += 1
        except Exception as e:
            print(f"❌ 错误跳过 {expiry} {strike}: {e}")
            fail += 1

    print(f"\n📦 保存完成：成功 {success} 个，失败 {fail} 个，输出目录：{output_dir}")

if __name__ == "__main__":
    CONFIG = {
        'ticker': ticker_base,
        'call_or_put': 'C',
        'strikes': list(range(200, 290, 5)),
        'start_date': "2024-07-01",
        'end_date': "2025-07-01"
    }

    selected = find_full_coverage_options(
        ticker=CONFIG['ticker'],
        start_date=CONFIG['start_date'],
        end_date=CONFIG['end_date'],
        strikes=CONFIG['strikes'],
        call_or_put=CONFIG['call_or_put'],
        min_contracts=500
    )

    save_option_data_to_csv(
        ticker_base=CONFIG['ticker'],
        options=selected,
        call_or_put=CONFIG['call_or_put'],
        start_date=CONFIG['start_date'],
        end_date=CONFIG['end_date'],
        output_dir="ticker_hedge_data"
    )